In [1]:
# Install dependencies
try:
    import transformer_lens
except:
    %pip install einops datasets jaxtyping "sae_lens>=3.23.1" openai tabulate umap-learn hdbscan eindex-callum git+https://github.com/callummcdougall/CircuitsVis.git#subdirectory=python git+https://github.com/callummcdougall/sae_vis.git@callum/v3 "transformer-lens>=2.7.0"


In [2]:
import gc
import itertools
import math
import os
import random
import sys
from collections import Counter, defaultdict
from copy import deepcopy
from dataclasses import dataclass
from functools import partial
from pathlib import Path
from typing import Any, Callable, Literal, TypeAlias

import circuitsvis as cv
import einops
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import requests
import torch as t
from datasets import load_dataset
from huggingface_hub import hf_hub_download
from IPython.display import HTML, IFrame, clear_output, display
from jaxtyping import Float, Int
from openai import OpenAI
from rich import print as rprint
from rich.table import Table
from sae_lens import (
    SAE,
    ActivationsStore,
    HookedSAETransformer,
    LanguageModelSAERunnerConfig,
    SAEConfig,
    SAETrainingRunner,
    upload_saes_to_huggingface,
)
from sae_lens.toolkit.pretrained_saes_directory import get_pretrained_saes_directory
from sae_vis import SaeVisConfig, SaeVisData, SaeVisLayoutConfig
from tabulate import tabulate
from torch import Tensor, nn
from torch.distributions.categorical import Categorical
from torch.nn import functional as F
from tqdm.auto import tqdm
from transformer_lens import ActivationCache, HookedTransformer
from transformer_lens.hook_points import HookPoint
from transformer_lens.utils import get_act_name, test_prompt, to_numpy

device = t.device("mps" if t.backends.mps.is_available() else "cuda" if t.cuda.is_available() else "cpu")

MAIN = __name__ == "__main__"

In [3]:
gpt2 = HookedSAETransformer.from_pretrained("gpt2-small", device=device)
layer = 9

gpt2_saes = {
    layer: SAE.from_pretrained(
        release="gpt2-small-res-jb",
        sae_id=f"blocks.{layer}.hook_resid_pre",
        device=str(device),
    )[0]
    for layer in tqdm(range(gpt2.cfg.n_layers))
}

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

sae_weights.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

sparsity.safetensors:   0%|          | 0.00/98.4k [00:00<?, ?B/s]

cfg.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

sae_weights.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

sparsity.safetensors:   0%|          | 0.00/98.4k [00:00<?, ?B/s]

In [4]:
GENERATE_KWARGS = dict(temperature=0.5, freq_penalty=2.0, verbose=False)
prompt = "The Vellore Institute of Technology is"

In [5]:
# normal generation of text

output = gpt2.generate(prompt, max_new_tokens = 30, **GENERATE_KWARGS)
output

"The Vellore Institute of Technology is a nonprofit organization that works to advance the understanding of the human body. The institute's mission is to provide information and resources for people with disabilities, including"

In [67]:
latent = {
    'software': 225,
    'crime': 8402,
    'legal': 22423,
    'death': 7853,
    'abuse': 11519,
    'military': 3577,
    'disaster': 3017,
    'deception': 19845,
    'suicide': 24138,
    'harm': 6911,
    # 'deception': 2130  # this appears to be more politics oriented
}

In [15]:
def steering_hook(
    activations: Float[Tensor, "batch pos d_in"],
    hook: HookPoint,
    sae: SAE,
    latent_idx: int,
    steering_coefficient: float,
) -> Tensor:
    """
    Steers the model by returning a modified activations tensor, with some multiple of the steering vector added to all
    sequence positions.
    """
    return activations + steering_coefficient * sae.W_dec[latent_idx]


def generate_with_steering(
    model: HookedSAETransformer,
    sae: SAE,
    prompt: str,
    latent_idx: int,
    steering_coefficient: float = 1.0,
    max_new_tokens: int = 50,
):
    """
    Generates text with steering. A multiple of the steering vector (the decoder weight for this latent) is added to
    the last sequence position before every forward pass.
    """
    _steering_hook = partial(
        steering_hook,
        sae=sae,
        latent_idx=latent_idx,
        steering_coefficient=steering_coefficient,
    )

    with model.hooks(fwd_hooks=[(sae.cfg.hook_name, _steering_hook)]):
        output = model.generate(prompt, max_new_tokens=max_new_tokens, **GENERATE_KWARGS)

    return output


In [12]:
# steering the latent involved during criminal police reporting

generate_with_steering(
    model = gpt2,
    sae = gpt2_saes[layer],
    prompt = "The Vellore Institute of Technology is",
    latent_idx = latent['crime'],
    steering_coefficient = 70.0,
    max_new_tokens = 60
)

'The Vellore Institute of Technology is a university in downtown Los Angeles about 30 miles from the University of California-San Francisco. The school was founded by a 17-year standing at a 6-0 hit taken by an out riding on the back riding thrown by one of his teammates. Born with pronounced left handed found not breathing, he'

In [13]:
# steering the latent involved in legality

generate_with_steering(
    model = gpt2,
    sae = gpt2_saes[layer],
    prompt = "The Vellore Institute of Technology is",
    latent_idx = latent['legal'],
    steering_coefficient = 70.0,
    max_new_tokens = 60
)

'The Vellore Institute of Technology is advising a challenge to the status quo by challenging the definition of marriage and redefising definitions for marriage.\n\nThe ruling will be heard in a process that could take years, experts said. The legal framework is complex, but it would be difficult to ensure that all cases are fully understood before the decision'

In [16]:
# steering the latent related to concepts of death

generate_with_steering(
    model = gpt2,
    sae = gpt2_saes[layer],
    prompt = "The Vellore Institute of Technology is",
    latent_idx = latent['death'],
    steering_coefficient = 70.0,
    max_new_tokens = 60
)

'The Vellore Institute of Technology is a research and education institution that specializes in the study and prevention of cancer. The goal is to inform, educate, inspire, inspire and prepare for the inevitable of it.\n\nIn March 2014, at the age of 30, I was on a journey that would have made me by far my greatest'

In [19]:
# steering the latent related to concepts of abuse

generate_with_steering(
    model = gpt2,
    sae = gpt2_saes[layer],
    prompt = "Hi, I am Devesh Ahuja, ",
    latent_idx = latent['abuse'],
    steering_coefficient = 70.0,
    max_new_tokens = 60
)

'Hi, I am Devesh Ahuja, ive been abused by my wife and child. The abuse is happening at a rate of almost every man. My wife was as a child and my son is as an adult. At the time of this abuse I had no memory or memory of any other children or women in my family. It was very'

In [22]:
# REVERSING the steering the latent related to concepts of abuse

generate_with_steering(
    model = gpt2,
    sae = gpt2_saes[layer],
    prompt = "Hi, I am Devesh Ahuja, ive been abused by my wife and child.",
    latent_idx = latent['abuse'],
    steering_coefficient = -30.0,
    max_new_tokens = 60
)

'Hi, I am Devesh Ahuja, ive been abused by my wife and child. We have two daughters 18-24 years old and 16-18 years old. We are living in a small house on the outskirts of Mumbai with our parents who live in a small city. We have been living here for 3 months now and we are happy to get married soon after we get married.'

In [29]:
# steering the latent related to terms of military stratergy and maneuvers

generate_with_steering(
    model = gpt2,
    sae = gpt2_saes[layer],
    prompt = "Hi, I am Devesh Ahuja, ",
    latent_idx = latent['military'],
    steering_coefficient = 70.0,
    max_new_tokens = 60
)

'Hi, I am Devesh Ahuja, \xa0and I have a few points to make.\n1. The first thing you need is a large force to break the front line and close the retreat. If your retreat is not under flank and you are behind an enemy position on a flank, your shot will be too close to his position and'

In [31]:
# steering the latent related to terms of military stratergy and maneuvers

generate_with_steering(
    model = gpt2,
    sae = gpt2_saes[layer],
    prompt = "This is Devesh Ahuja, ",
    latent_idx = latent['disaster'],
    steering_coefficient = 70.0,
    max_new_tokens = 60
)

'This is Devesh Ahuja, Â the deceased victim of a life-threatening injury to the head of an 18-year-old man and a minor one to two others. The local one was injured in the initial emergency and has been transported to hospital for critical injuries.\n\nThe three lives that were lost to serious injuries have'

In [53]:
# steering the latent related to deception

generate_with_steering(
    model = gpt2,
    sae = gpt2_saes[layer],
    prompt = "Meet Devesh Ahuja, ",
    latent_idx = latent['deception'],
    steering_coefficient = 70.0,
    max_new_tokens = 80
)

'Meet Devesh Ahuja, \xa0a passionate advocate for the welfare of the world. "I am a lawyer and legal expert." "I\'m a humanitarian, but I\'m not doing anything to help people because it\'s just \'just doing good.\' But I do care for the health of the people. It\'s all about being concerned about their wellbeing." All that is helping to protect them, but there are some fundamental issues that'

In [66]:
# steering the latent related to suicide

generate_with_steering(
    model = gpt2,
    sae = gpt2_saes[layer],
    prompt = "Meet Devesh Ahuja, ",
    latent_idx = latent['suicide'],
    steering_coefficient = 70.0,
    max_new_tokens = 80
)

"Meet Devesh Ahuja, \xa0a young man who was recently involved in a suicide attempt.\nI was planning to be a coach for my brother and I had been thinking about it. At the time I thought that suicide would be an option because of the way it is and I wanted to try out as many different types of methods as possible. It's not that I didn't want to try out some kind of method but"